In [2]:

from pathlib import Path
import os
from subprocess import Popen, STDOUT
import scanpy as sc

# harcoded configs
recompute = True

current_folder = globals()['_dh'][0] # for jupyter notebook
#current_folder = Path(__file__).parent
raw_location = current_folder / ".." / ".." / "data" / "raw_old_sn"
filtered_location = current_folder / ".." / ".." / "data" / "raw" / "visium"
output_dir = current_folder / ".." / ".." / "data" / "cellbender_out"

# verbose, helpful for debugging
print("recompute: ", recompute)
print("raw_location: ", raw_location)
print("filtered_location: ", filtered_location)
print("output_dir: ", output_dir)

samples = [sample for sample in os.listdir(raw_location) if not sample.startswith(".")]

recompute:  True
raw_location:  /home/philipp/Work/VisiumMS/scripts/process/../../data/raw_old_sn
filtered_location:  /home/philipp/Work/VisiumMS/scripts/process/../../data/raw/visium
output_dir:  /home/philipp/Work/VisiumMS/scripts/process/../../data/cellbender_out


In [4]:
sample = samples[0]
sample

'MS411'

In [5]:
output_dir_sample = output_dir / sample
output_dir_sample.mkdir(parents=True, exist_ok=True)
print(output_dir_sample)

/home/philipp/Work/VisiumMS/scripts/process/../../data/cellbender_out/MS411


In [8]:
# TODO: Here using as expected cell number the filtered barcode matrix
filtered_adata = sc.read_10x_h5(filtered_location / sample / "outs" / "filtered_feature_bc_matrix.h5")
expected_cells = filtered_adata.n_obs
del filtered_adata
expected_cells

/home/philipp/miniconda3/envs/torch_env2/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


2826

In [9]:
input_h5 = raw_location / sample / "raw_feature_bc_matrix.h5"
print(input_h5)

/home/philipp/Work/VisiumMS/scripts/process/../../data/raw_old_sn/MS411/raw_feature_bc_matrix.h5


In [10]:
test = sc.read_10x_h5(input_h5)

/home/philipp/miniconda3/envs/torch_env2/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [23]:
p = Popen(["cellbender", "remove-background", 
    "--input", str(input_h5), # input file
    "--output", "cell_bender_matrix.h5",
    "--model", "full",
    "--cuda", # cuda enables
    "--expected-cells", str(expected_cells), # number of expected cells, no default
    "--total-droplets-included", str(25000), # number of droplets from the rank-ordered UMI plot that will be analyzed, default 25_000
    "--fpr", str(0.01), # target false positive rate, default 0.01
    "--epochs", str(150), # how many epochs to train, default 150
    "--posterior-batch-size",  str(5), # batch size for creating the posterior, default 20 (important if running of of GPU RAM)
    "--cells-posterior-reg-calc", str(50)], # number of cells used to estimate posterior regularization lambda, default 100
    cwd=str(output_dir_sample),
    stderr=STDOUT)

p.wait(timeout=None)  

cellbender:remove-background: Command:
cellbender remove-background --input /home/philipp/Work/VisiumMS/scripts/process/../../data/raw_old_sn/MS411/raw_feature_bc_matrix.h5 --output cell_bender_matrix.h5 --model full --cuda --expected-cells 2826 --total-droplets-included 25000 --fpr 0.01 --epochs 150 --posterior-batch-size 5 --cells-posterior-reg-calc 50
cellbender:remove-background: 2023-05-30 12:45:35
cellbender:remove-background: Running remove-background
cellbender:remove-background: Loading data from file /home/philipp/Work/VisiumMS/scripts/process/../../data/raw_old_sn/MS411/raw_feature_bc_matrix.h5
cellbender:remove-background: CellRanger v3 format
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Including 30689 genes that have nonzero counts.
/home/philipp/Work/CellBender/cellbender/remove_background/data/dataset.py:1436: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically

KeyboardInterrupt: 

cellbender:remove-background: [epoch 001]  average training loss: 2872.6945
cellbender:remove-background: [epoch 002]  average training loss: 2382.3919  (17.5 seconds per epoch)
cellbender:remove-background: [epoch 003]  average training loss: 2276.4157
cellbender:remove-background: [epoch 004]  average training loss: 2224.0942
cellbender:remove-background: [epoch 005]  average training loss: 2205.7675
cellbender:remove-background: [epoch 005] average test loss: 2160.3754
cellbender:remove-background: [epoch 006]  average training loss: 2186.7893
cellbender:remove-background: [epoch 007]  average training loss: 2172.8793
cellbender:remove-background: [epoch 008]  average training loss: 2160.5785
cellbender:remove-background: [epoch 009]  average training loss: 2159.7639
cellbender:remove-background: [epoch 010]  average training loss: 2153.3295
cellbender:remove-background: [epoch 010] average test loss: 2124.8041
cellbender:remove-background: [epoch 011]  average training loss: 2144.0

In [ ]:


output_dir_sample = output_dir / sample
output_dir_sample.mkdir(parents=True, exist_ok=True)
print(output_dir_sample)



# TODO: Here using as expected cell number the filtered barcode matrix
filtered_adata = sc.read_10x_h5(filtered_location / sample / "outs" / "filtered_feature_bc_matrix.h5")
expected_cells = filtered_adata.n_obs
del filtered_adata

input_h5 = raw_location / sample / "raw_feature_bc_matrix.h5"
print(input_h5)

# create log file
log_file = output_dir / sample / "cell_bender_custom_log.txt"
with open(log_file, "w") as f:

    f.write("input_h5: " + str(input_h5) + "\n")

    f.write("output_dir_sample: " + str(output_dir_sample) + "\n")

    p = Popen(["cellbender", "remove-background", 
        "--input", input_h5, # input file
        "--output", "cell_bender_matrix.h5",
        "--model", "full",
        "--cuda", # cuda enables
        "--expected-cells", str(expected_cells), # number of expected cells, no default
        "--total-droplets-included", str(25000), # number of droplets from the rank-ordered UMI plot that will be analyzed, default 25_000
        "--fpr", str(0.01), # target false positive rate, default 0.01
        "--epochs", str(150), # how many epochs to train, default 150
        "--posterior-batch-size",  str(5), # batch size for creating the posterior, default 20 (important if running of of GPU RAM)
        "--cells-posterior-reg-calc", str(50)], # number of cells used to estimate posterior regularization lambda, default 100
        cwd=output_dir_sample,
        stderr=STDOUT,
        stdout=f
        )
    
    p.wait(timeout=None)    # https://docs.python.org/3/library/subprocess.html#subprocess.Popen.wait
                            # don't spawn another process if p has not finished yet (sequential handling)

# install cellbender: pip install -e .
# call: python scripts/process/cell_bender.py 